In [14]:
!pip install neo4j


[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
import glob

with open("elt_cypher_queries/cleanup_nodes.cypher") as f:
    cleanup_node_queries = [line for line in f]

with open("elt_cypher_queries/cleanup_edges.cypher") as f:
    cleanup_edge_queries = [line for line in f]

with open("elt_cypher_queries/cleanup_constraints.cypher") as f:
    cleanup_constraints_queries = [line for line in f]

with open("elt_cypher_queries/index_nodes.cypher") as f:
    node_index_queries = [line for line in f]

node_cypher_files = glob.glob("elt_cypher_queries/nodes/*.cypher")
node_creation_queries = []
for query_file in node_cypher_files:
    with open(query_file) as f:
        creation_query = f.read()
        node_creation_queries.append(creation_query)

edge_cypher_files = glob.glob("elt_cypher_queries/edges/*.cypher")
edge_creation_queries = []
for query_file in edge_cypher_files:
    with open(query_file) as f:
        creation_query = f.read()
        edge_creation_queries.append(creation_query)

In [19]:
from neo4j import GraphDatabase

def run_my_query(tx):
        result = tx.run(query)
        return list(result)

def execute_query(query, uri, user, password):

    with  GraphDatabase.driver(uri, auth=(user, password)) as driver:
         driver.verify_connectivity()
         print("Connection estabilished.")
         with driver.session(database="neo4j") as session:
            result = session.run(query)
            summary = result.consume()
            print("Created {nodes_created} nodes and {relationships_created} relationships in {time} ms.".format(
        nodes_created=summary.counters.nodes_created,
        relationships_created=summary.counters.relationships_created,
        time=summary.result_available_after))

     
for query in cleanup_edge_queries+cleanup_constraints_queries+cleanup_node_queries:
    print(f"Processing {query}")
    execute_query(query, "bolt://localhost:7687", "neo4j", "pass1234")

for query in node_creation_queries+node_index_queries:
    print(f"Processing {query}")
    execute_query(query, "bolt://localhost:7687", "neo4j", "pass1234")

for query in edge_creation_queries:
    print(f"Processing {query}")
    execute_query(query, "bolt://localhost:7687", "neo4j", "pass1234")


Processing MATCH ()-[r]->() CALL { WITH r DETACH DELETE r} IN TRANSACTIONS OF 1000000 ROWS;
Connection estabilished.
Created 0 nodes and 0 relationships in 138 ms.
Processing DROP CONSTRAINT imdb_attribute_attribute IF EXISTS;

Connection estabilished.
Created 0 nodes and 0 relationships in 1 ms.
Processing DROP CONSTRAINT imdb_characters_somecharacter IF EXISTS;

Connection estabilished.
Created 0 nodes and 0 relationships in 1 ms.
Processing DROP CONSTRAINT imdb_distribution_type_distribution_type IF EXISTS;

Connection estabilished.
Created 0 nodes and 0 relationships in 0 ms.
Processing DROP CONSTRAINT imdb_genre_genre IF EXISTS;

Connection estabilished.
Created 0 nodes and 0 relationships in 1 ms.
Processing DROP CONSTRAINT imdb_job_category_category IF EXISTS;

Connection estabilished.
Created 0 nodes and 0 relationships in 9 ms.
Processing DROP CONSTRAINT imdb_language_code_alpha2 IF EXISTS;

Connection estabilished.
Created 0 nodes and 0 relationships in 1 ms.
Processing DROP 

In [ ]:
# for query in cleanup_edge_queries+edge_creation_queries:
#     print(f"Processing {query}")
#     loader = IMDBELTLoader("bolt://localhost:7687", "neo4j", "pass1234")
#     loader.execute_query(query)
#     loader.close()